# Data Cleaning

In [1]:
import pandas as pd
import datetime

## Housing Data

### Importing Data

In [2]:
house_df = pd.read_csv("sourced_data/Melbourne_housing_FULL.csv")
house_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


### Removing Unneeded Columns

In [3]:
# Obtaining the column names for the dataframe
house_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [4]:
# Checking the values in the Type column
house_df["Type"].unique()

array(['h', 'u', 't'], dtype=object)

In [5]:
# Checking the number of suburbs in the dataset
house_df["Suburb"].value_counts()

Reservoir         844
Bentleigh East    583
Richmond          552
Glen Iris         491
Preston           485
                 ... 
Ferny Creek         1
Eynesbury           1
Hopetoun Park       1
Avonsleigh          1
Olinda              1
Name: Suburb, Length: 351, dtype: int64

In [6]:
house_suburbs = house_df["Suburb"].unique()
len(house_suburbs)

351

In [7]:
# Checking the number of suburbs in the dataset
house_df["Postcode"].value_counts()

3073.0    844
3046.0    638
3020.0    617
3121.0    612
3165.0    583
         ... 
3139.0      1
3159.0      1
3793.0      1
3788.0      1
3428.0      1
Name: Postcode, Length: 211, dtype: int64

#### What should be included?

As the model will predict property prices for listings supplied via domain.com.au, the columns used to train the model must be available from the listings. This will allow the model to input all required features when creating a prediction. The included columns will also need to be features that can be reasonably assumed to influence property prices.

The model will include the following features from this dataset:
   - Number of Rooms
   - Property Type: House, Unit, Townhouse
   - The distance from the suburb to the CBD
   - Number of Bathrooms
   - Number of Car spaces
   - The Landsize of the property
   - The Year the house was sold
   - The Month the house was sold

#### Location

Suburb and Postcode were considered for the model, however it was decided that they would not be included.

Firstly, Postcode is categorical data and would need to be treated accordingly. Therefore, it would need to be binary encoded, creating a great number of extra columns in the data set. As Postcode would not be providing any information that greatly differs from Suburb, which would also need to be binary encoded, it was decided that it would not be included.

Secondly, Suburb was highlighted as a feature thought to be important for the model and price prediction. However, as it is categorical, it would need to be binary encoded, adding 351 extra columns to the dataframe. However, due to high cardinality, encoding the Suburbs in this way would not add value to the model. Especially considering some suburbs have a much greater presence in the model than others, the model would not be able to learn pricing information based on location in this way.

It was therefore determined that Distance would be used as the data which represents the location of the property.

Note: Suburb will be kept in the data for now as it will need to be used as a merging column later.

#### Rooms

The data has two values that represent the number of rooms in a property: Rooms and Bedroom2.
The Kaggle source explained that the data in these columns was obtained from two different data sources.
The best column to use will therefore neeed to be determined.

In [8]:
# Checking the difference between Rooms and Bedroom2 by counting the NaN values in each column. 

house_df[['Rooms', 'Bedroom2']].isna().sum()

Rooms          0
Bedroom2    8217
dtype: int64

The Rooms column has no NaN values whereas the Bedroom2 column has over 8000, therefore the Bedroom2 column will be dropped.

#### Dropping columns

In [9]:
house_df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


In [10]:
# Obtaining the column names for the dataframe
house_df.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [11]:
# Dropping unneeded columns

house_df = house_df.drop(columns=['Address', 'Method', 'SellerG', 'Postcode', 'Bedroom2', 'BuildingArea', 'YearBuilt',
                                  'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount'])
house_df.head()

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,Landsize
0,Abbotsford,2,h,NaN,3/09/2016,2.5,1.0,1.0,126.0
1,Abbotsford,2,h,1480000.0,3/12/2016,2.5,1.0,1.0,202.0
2,Abbotsford,2,h,1035000.0,4/02/2016,2.5,1.0,0.0,156.0
3,Abbotsford,3,u,NaN,4/02/2016,2.5,2.0,1.0,0.0
4,Abbotsford,3,h,1465000.0,4/03/2017,2.5,2.0,0.0,134.0


### Removing NaNs


In [12]:
# Identifying NaN values
house_df.isna().sum()

Suburb          0
Rooms           0
Type            0
Price        7610
Date            0
Distance        1
Bathroom     8226
Car          8728
Landsize    11810
dtype: int64

In [13]:
# Drop the rows that contain null values
house_df = house_df.dropna()
house_df

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,Landsize
1,Abbotsford,2,h,1480000.0,3/12/2016,2.5,1.0,1.0,202.0
2,Abbotsford,2,h,1035000.0,4/02/2016,2.5,1.0,0.0,156.0
4,Abbotsford,3,h,1465000.0,4/03/2017,2.5,2.0,0.0,134.0
5,Abbotsford,3,h,850000.0,4/03/2017,2.5,2.0,1.0,94.0
6,Abbotsford,4,h,1600000.0,4/06/2016,2.5,1.0,2.0,120.0
...,...,...,...,...,...,...,...,...,...
34849,Wollert,3,h,570000.0,24/02/2018,25.5,2.0,2.0,404.0
34852,Yarraville,4,h,1480000.0,24/02/2018,6.3,1.0,3.0,593.0
34853,Yarraville,2,h,888000.0,24/02/2018,6.3,2.0,1.0,98.0
34854,Yarraville,2,t,705000.0,24/02/2018,6.3,1.0,2.0,220.0


### Separating Date into Year and Month columns

In [14]:
# Adding a 'Year' column
house_df['Year'] = pd.DatetimeIndex(house_df['Date']).year
house_df.head()

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,Landsize,Year
1,Abbotsford,2,h,1480000.0,3/12/2016,2.5,1.0,1.0,202.0,2016
2,Abbotsford,2,h,1035000.0,4/02/2016,2.5,1.0,0.0,156.0,2016
4,Abbotsford,3,h,1465000.0,4/03/2017,2.5,2.0,0.0,134.0,2017
5,Abbotsford,3,h,850000.0,4/03/2017,2.5,2.0,1.0,94.0,2017
6,Abbotsford,4,h,1600000.0,4/06/2016,2.5,1.0,2.0,120.0,2016


In [15]:
# Determining the years in the dataset 
house_df["Year"].unique()

array([2016, 2017, 2018])

In [16]:
# Adding a 'Month' column
house_df['Month'] = pd.DatetimeIndex(house_df['Date']).month
house_df.head()

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,Landsize,Year,Month
1,Abbotsford,2,h,1480000.0,3/12/2016,2.5,1.0,1.0,202.0,2016,3
2,Abbotsford,2,h,1035000.0,4/02/2016,2.5,1.0,0.0,156.0,2016,4
4,Abbotsford,3,h,1465000.0,4/03/2017,2.5,2.0,0.0,134.0,2017,4
5,Abbotsford,3,h,850000.0,4/03/2017,2.5,2.0,1.0,94.0,2017,4
6,Abbotsford,4,h,1600000.0,4/06/2016,2.5,1.0,2.0,120.0,2016,4


In [17]:
# Determining the datatype of the month column
house_df["Month"].dtype

dtype('int64')

In [18]:
# subtracting one from each month so that it starts with January as 0, all the way through to December as 11

house_df["Month"] = house_df["Month"].sub(1)
house_df.head()

,Suburb,Rooms,Type,Price,Date,Distance,Bathroom,Car,Landsize,Year,Month
1,Abbotsford,2,h,1480000.0,3/12/2016,2.5,1.0,1.0,202.0,2016,2
2,Abbotsford,2,h,1035000.0,4/02/2016,2.5,1.0,0.0,156.0,2016,3
4,Abbotsford,3,h,1465000.0,4/03/2017,2.5,2.0,0.0,134.0,2017,3
5,Abbotsford,3,h,850000.0,4/03/2017,2.5,2.0,1.0,94.0,2017,3
6,Abbotsford,4,h,1600000.0,4/06/2016,2.5,1.0,2.0,120.0,2016,3


In [19]:
# Removing the date column
house_df = house_df.drop(columns=['Date'])
house_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month
1,Abbotsford,2,h,1480000.0,2.5,1.0,1.0,202.0,2016,2
2,Abbotsford,2,h,1035000.0,2.5,1.0,0.0,156.0,2016,3
4,Abbotsford,3,h,1465000.0,2.5,2.0,0.0,134.0,2017,3
5,Abbotsford,3,h,850000.0,2.5,2.0,1.0,94.0,2017,3
6,Abbotsford,4,h,1600000.0,2.5,1.0,2.0,120.0,2016,3


In [20]:
# Exporting the dataframe as html script
house_df.to_html('output_data/house.html', index = False)

## Crime Data

To add an extra feature to the model, crime data will be included. The data will be the number of crimes commited in a specific suburb over the span of a year.

### Importing Data

In [21]:
crime_df = pd.read_csv("sourced_data/CrimeSuburbYear.csv")
crime_df.head()

,postcode,suburb,lat,lon,Local Government Area,Region,Year,A20,A50,A70,...,F20,F30,F90,Total,A,B,C,D,E,F
0,3000,melbourne,-37.814563,144.970267,Melbourne,Northern Metropolitan,2011,1032,116,99,...,13,36,3,14175,1414,7331,404,3764,1210,52
1,3002,east melbourne,-37.816640,144.987811,Melbourne,Northern Metropolitan,2011,53,12,4,...,0,9,0,753,76,476,32,149,11,9
2,3003,west melbourne,-37.806255,144.941123,Melbourne,Northern Metropolitan,2011,54,9,3,...,2,1,2,633,80,403,32,107,6,5
3,3006,southbank,-37.823258,144.965926,Melbourne,Southern Metropolitan,2011,237,21,14,...,0,14,2,2059,310,1103,60,545,25,16
4,3008,docklands,-37.814719,144.948039,Melbourne,Southern Metropolitan,2011,113,7,8,...,4,6,3,1244,149,641,35,389,17,13


In [22]:
# Determining the column names for the dataset
crime_df.columns

Index(['postcode', 'suburb', 'lat', 'lon', 'Local Government Area', 'Region',
       'Year', 'A20', 'A50', 'A70', 'A80', 'A90', 'B10', 'B20', 'B30', 'B40',
       'B50', 'B60', 'C10', 'C20', 'C30', 'C90', 'D10', 'D20', 'D30', 'D40',
       'E10', 'E20', 'F10', 'F20', 'F30', 'F90', 'Total', 'A', 'B', 'C', 'D',
       'E', 'F'],
      dtype='object')

### Removing Unneeded Columns

The only columns required are Suburb, Year and Total.

Suburb will be used to join this dataset with the housing data.

Year will allow the crime statistics to be assinged to the correct property sales in the housing dataframe, based on the year the property was sold.

Total is the total number of crimes commited in a suburb in the given year. It is the addition of the 'A', 'B', 'C', 'D', 'E' and 'F' columns, where these represent types of crimes.

In [23]:
# Specifying columns that are being kept
crime_df = crime_df[['suburb', 'Year', 'Total']]
crime_df.head()

,suburb,Year,Total
0,melbourne,2011,14175
1,east melbourne,2011,753
2,west melbourne,2011,633
3,southbank,2011,2059
4,docklands,2011,1244


#### Creating a dataframe specifically for the website

In [24]:
# Creating 2016, 2017, 2018 dataframe
crime_16_17_18_df = crime_df.loc[(crime_df["Year"] == 2016) | (crime_df["Year"] == 2017) | (crime_df["Year"] == 2018), :]
crime_16_17_18_df.head()

,suburb,Year,Total
14135,melbourne,2016,15485
14136,east melbourne,2016,818
14137,west melbourne,2016,705
14138,southbank,2016,2197
14139,docklands,2016,1578


In [25]:
# Updating column names
crime_16_17_18_df = crime_16_17_18_df.rename(columns = {"suburb" : "Suburb", "Total" : "Number of Crimes"})
crime_16_17_18_df.head()

,Suburb,Year,Number of Crimes
14135,melbourne,2016,15485
14136,east melbourne,2016,818
14137,west melbourne,2016,705
14138,southbank,2016,2197
14139,docklands,2016,1578


In [26]:
# Exporting the dataframe as a html script
crime_16_17_18_df.to_html('output_data/crime.html', index = False)

### Separating the data into required years

Performing a loc for each year to separate the data.

#### 2016

In [27]:
# Selecting 2016 data
crime_2016_df = crime_df.loc[(crime_df["Year"] == 2016), :]
crime_2016_df.head()

,suburb,Year,Total
14135,melbourne,2016,15485
14136,east melbourne,2016,818
14137,west melbourne,2016,705
14138,southbank,2016,2197
14139,docklands,2016,1578


In [28]:
# Updating column names
crime_2016_df = crime_2016_df.rename(columns = {"suburb" : "Suburb", "Total" : "Total: 2016"})
crime_2016_df.head()

,Suburb,Year,Total: 2016
14135,melbourne,2016,15485
14136,east melbourne,2016,818
14137,west melbourne,2016,705
14138,southbank,2016,2197
14139,docklands,2016,1578


In [29]:
# Removing year column
crime_2016_df = crime_2016_df[["Suburb", "Total: 2016"]]
crime_2016_df.head()

,Suburb,Total: 2016
14135,melbourne,15485
14136,east melbourne,818
14137,west melbourne,705
14138,southbank,2197
14139,docklands,1578


#### 2017

In [30]:
# Selecting 2017 data
crime_2017_df = crime_df.loc[(crime_df["Year"] == 2017), :]
crime_2017_df.head()

,suburb,Year,Total
16962,melbourne,2017,15162
16963,east melbourne,2017,666
16964,west melbourne,2017,743
16965,southbank,2017,2309
16966,docklands,2017,1777


In [31]:
# Updating column names
crime_2017_df = crime_2017_df.rename(columns = {"suburb" : "Suburb", "Total" : "Total: 2017"})
crime_2017_df.head()

,Suburb,Year,Total: 2017
16962,melbourne,2017,15162
16963,east melbourne,2017,666
16964,west melbourne,2017,743
16965,southbank,2017,2309
16966,docklands,2017,1777


In [32]:
# Removing year column
crime_2017_df = crime_2017_df[["Suburb", "Total: 2017"]]
crime_2017_df.head()

,Suburb,Total: 2017
16962,melbourne,15162
16963,east melbourne,666
16964,west melbourne,743
16965,southbank,2309
16966,docklands,1777


#### 2018

In [33]:
# Selecting 2018 data
crime_2018_df = crime_df.loc[(crime_df["Year"] == 2018), :]
crime_2018_df.head()

,suburb,Year,Total
19789,melbourne,2018,14780
19790,east melbourne,2018,782
19791,west melbourne,2018,864
19792,southbank,2018,2476
19793,docklands,2018,2317


In [34]:
# Updating column names
crime_2018_df = crime_2018_df.rename(columns = {"suburb" : "Suburb", "Total" : "Total: 2018"})
crime_2018_df.head()

,Suburb,Year,Total: 2018
19789,melbourne,2018,14780
19790,east melbourne,2018,782
19791,west melbourne,2018,864
19792,southbank,2018,2476
19793,docklands,2018,2317


In [35]:
# Removing year column
crime_2018_df = crime_2018_df[["Suburb", "Total: 2018"]]
crime_2018_df.head()

,Suburb,Total: 2018
19789,melbourne,14780
19790,east melbourne,782
19791,west melbourne,864
19792,southbank,2476
19793,docklands,2317


#### Isolating the 2020 values so this can be used alongside the property listing input

In [36]:
# Selecting 2020 data
crime_2020_df = crime_df.loc[(crime_df["Year"] == 2020), :]
crime_2020_df.head()

,suburb,Year,Total
25443,melbourne,2020,14174
25444,east melbourne,2020,907
25445,west melbourne,2020,1082
25446,southbank,2020,2794
25447,docklands,2020,2439


In [37]:
# Updating column names
crime_2020_df = crime_2020_df.rename(columns = {"suburb" : "Suburb", "Total" : "Total: 2020"})
crime_2020_df.head()

,Suburb,Year,Total: 2020
25443,melbourne,2020,14174
25444,east melbourne,2020,907
25445,west melbourne,2020,1082
25446,southbank,2020,2794
25447,docklands,2020,2439


In [38]:
# Removing year column
crime_2020_df = crime_2020_df[["Suburb", "Total: 2020"]]
crime_2020_df.head()

,Suburb,Total: 2020
25443,melbourne,14174
25444,east melbourne,907
25445,west melbourne,1082
25446,southbank,2794
25447,docklands,2439


In [39]:
# Exporting to CSV
crime_2020_df.to_csv('output_data/crime_2020_df.csv', index = False)


## Joining House and Crime Data

### Changing the representation of the suburb values

The two dataframes will be joined on the suburb column. Before this is possible, the dataframes will need to be adjusted as the housing data has the suburbs capitalised, whereas the crime data does not.

In [40]:
# Setting the Suburbs in the housing data as lowercase
house_df['Suburb'] = house_df['Suburb'].str.lower()
house_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month
1,abbotsford,2,h,1480000.0,2.5,1.0,1.0,202.0,2016,2
2,abbotsford,2,h,1035000.0,2.5,1.0,0.0,156.0,2016,3
4,abbotsford,3,h,1465000.0,2.5,2.0,0.0,134.0,2017,3
5,abbotsford,3,h,850000.0,2.5,2.0,1.0,94.0,2017,3
6,abbotsford,4,h,1600000.0,2.5,1.0,2.0,120.0,2016,3


### Separating the house data into required years

Performing a loc for each year to separate the data.

#### 2016

In [41]:
# Selecting 2016 data
house_2016_df = house_df.loc[(house_df["Year"] == 2016), :]
house_2016_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month
1,abbotsford,2,h,1480000.0,2.5,1.0,1.0,202.0,2016,2
2,abbotsford,2,h,1035000.0,2.5,1.0,0.0,156.0,2016,3
6,abbotsford,4,h,1600000.0,2.5,1.0,2.0,120.0,2016,3
10,abbotsford,2,h,941000.0,2.5,1.0,0.0,181.0,2016,6
11,abbotsford,3,h,1876000.0,2.5,2.0,0.0,245.0,2016,6


#### 2017

In [42]:
# Selecting 2017 data
house_2017_df = house_df.loc[(house_df["Year"] == 2017), :]
house_2017_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month
4,abbotsford,3,h,1465000.0,2.5,2.0,0.0,134.0,2017,3
5,abbotsford,3,h,850000.0,2.5,2.0,1.0,94.0,2017,3
38,abbotsford,4,h,1330000.0,2.5,2.0,2.0,780.0,2017,2
40,abbotsford,3,t,900000.0,2.5,2.0,2.0,0.0,2017,2
41,abbotsford,3,u,1090000.0,2.5,2.0,2.0,4290.0,2017,2


#### 2018

In [43]:
# Selecting 2018 data
house_2018_df = house_df.loc[(house_df["Year"] == 2018), :]
house_2018_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month
29219,airport west,3,h,970000.0,10.4,1.0,2.0,634.0,2018,5
29220,airport west,4,h,790000.0,10.4,2.0,1.0,585.0,2018,5
29221,airport west,3,h,1025000.0,10.4,1.0,1.0,791.0,2018,5
29227,altona,4,u,1160000.0,11.0,3.0,3.0,689.0,2018,5
29228,altona,3,h,925000.0,11.0,1.0,4.0,836.0,2018,5


### Merging the dataframe with the year appropriate crime data

#### 2016

In [44]:
# Merging 2016 data on suburb
house_crime_2016_df = house_2016_df.merge(crime_2016_df, how='left', on='Suburb')
house_crime_2016_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Total: 2016
0,abbotsford,2,h,1480000.0,2.5,1.0,1.0,202.0,2016,2,1053.0
1,abbotsford,2,h,1035000.0,2.5,1.0,0.0,156.0,2016,3,1053.0
2,abbotsford,4,h,1600000.0,2.5,1.0,2.0,120.0,2016,3,1053.0
3,abbotsford,2,h,941000.0,2.5,1.0,0.0,181.0,2016,6,1053.0
4,abbotsford,3,h,1876000.0,2.5,2.0,0.0,245.0,2016,6,1053.0


In [45]:
# Renaming crime column
house_crime_2016_df = house_crime_2016_df.rename(columns = {"Total: 2016" : "Crime"})
house_crime_2016_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Crime
0,abbotsford,2,h,1480000.0,2.5,1.0,1.0,202.0,2016,2,1053.0
1,abbotsford,2,h,1035000.0,2.5,1.0,0.0,156.0,2016,3,1053.0
2,abbotsford,4,h,1600000.0,2.5,1.0,2.0,120.0,2016,3,1053.0
3,abbotsford,2,h,941000.0,2.5,1.0,0.0,181.0,2016,6,1053.0
4,abbotsford,3,h,1876000.0,2.5,2.0,0.0,245.0,2016,6,1053.0


#### 2017

In [46]:
# Merging 2017 data on suburb
house_crime_2017_df = house_2017_df.merge(crime_2017_df, how='left', on='Suburb')
house_crime_2017_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Total: 2017
0,abbotsford,3,h,1465000.0,2.5,2.0,0.0,134.0,2017,3,1090.0
1,abbotsford,3,h,850000.0,2.5,2.0,1.0,94.0,2017,3,1090.0
2,abbotsford,4,h,1330000.0,2.5,2.0,2.0,780.0,2017,2,1090.0
3,abbotsford,3,t,900000.0,2.5,2.0,2.0,0.0,2017,2,1090.0
4,abbotsford,3,u,1090000.0,2.5,2.0,2.0,4290.0,2017,2,1090.0


In [47]:
# Renaming crime column
house_crime_2017_df = house_crime_2017_df.rename(columns = {"Total: 2017" : "Crime"})
house_crime_2017_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Crime
0,abbotsford,3,h,1465000.0,2.5,2.0,0.0,134.0,2017,3,1090.0
1,abbotsford,3,h,850000.0,2.5,2.0,1.0,94.0,2017,3,1090.0
2,abbotsford,4,h,1330000.0,2.5,2.0,2.0,780.0,2017,2,1090.0
3,abbotsford,3,t,900000.0,2.5,2.0,2.0,0.0,2017,2,1090.0
4,abbotsford,3,u,1090000.0,2.5,2.0,2.0,4290.0,2017,2,1090.0


#### 2018

In [48]:
# Merging 2018 data on suburb
house_crime_2018_df = house_2018_df.merge(crime_2018_df, how='left', on='Suburb')
house_crime_2018_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Total: 2018
0,airport west,3,h,970000.0,10.4,1.0,2.0,634.0,2018,5,812.0
1,airport west,4,h,790000.0,10.4,2.0,1.0,585.0,2018,5,812.0
2,airport west,3,h,1025000.0,10.4,1.0,1.0,791.0,2018,5,812.0
3,altona,4,u,1160000.0,11.0,3.0,3.0,689.0,2018,5,549.0
4,altona,3,h,925000.0,11.0,1.0,4.0,836.0,2018,5,549.0


In [49]:
# Renaming crime column
house_crime_2018_df = house_crime_2018_df.rename(columns = {"Total: 2018" : "Crime"})
house_crime_2018_df.head()

,Suburb,Rooms,Type,Price,Distance,Bathroom,Car,Landsize,Year,Month,Crime
0,airport west,3,h,970000.0,10.4,1.0,2.0,634.0,2018,5,812.0
1,airport west,4,h,790000.0,10.4,2.0,1.0,585.0,2018,5,812.0
2,airport west,3,h,1025000.0,10.4,1.0,1.0,791.0,2018,5,812.0
3,altona,4,u,1160000.0,11.0,3.0,3.0,689.0,2018,5,549.0
4,altona,3,h,925000.0,11.0,1.0,4.0,836.0,2018,5,549.0


### Creating one final dataframe

#### Adding dataframes to a dictionary

In [50]:
data_dict = {1: house_crime_2016_df, 2: house_crime_2017_df, 3: house_crime_2018_df}

#### Concatenating the dataframes

In [51]:
# Using pd.concat to join the dataframes in the dictionary
combined_data_df = pd.concat(data_dict)
combined_data_df

Suburb  Rooms Type      Price  Distance  Bathroom  Car  Landsize  \
1 0     abbotsford      2    h  1480000.0       2.5       1.0  1.0     202.0   
  1     abbotsford      2    h  1035000.0       2.5       1.0  0.0     156.0   
  2     abbotsford      4    h  1600000.0       2.5       1.0  2.0     120.0   
  3     abbotsford      2    h   941000.0       2.5       1.0  0.0     181.0   
  4     abbotsford      3    h  1876000.0       2.5       2.0  0.0     245.0   
...            ...    ...  ...        ...       ...       ...  ...       ...   
3 2181     wollert      3    h   570000.0      25.5       2.0  2.0     404.0   
  2182  yarraville      4    h  1480000.0       6.3       1.0  3.0     593.0   
  2183  yarraville      2    h   888000.0       6.3       2.0  1.0      98.0   
  2184  yarraville      2    t   705000.0       6.3       1.0  2.0     220.0   
  2185  yarraville      2    h  1020000.0       6.3       1.0  0.0     250.0   

        Year  Month   Crime  
1 0     2016      2  1053.0  
  1     2016      3  1053.0  
  2     2016      3  1053.0  
  3     2016      6  1053.0  
  4     2016      6  1053.0  
...      ...    ...     ...  
3 2181  2018      1   529.0  
  2182  2018      1   690.0  
  2183  2018      1   690.0  
  2184  2018      1   690.0  
  2185  2018      1   690.0  

[17701 rows x 11 columns]

#### Checking for null values

In [52]:
# Checking that no rows have been lost

print(f"The length of the house dataframe was {len(house_df.index)} rows.")
print(f"The length of the combined dataframe is {len(combined_data_df.index)} rows.")


The length of the house dataframe was 17701 rows.
The length of the combined dataframe is 17701 rows.


In [53]:
# Checking that combining the crime and house data hasn't resulted in null values
combined_data_df.isna().sum()

Suburb        0
Rooms         0
Type          0
Price         0
Distance      0
Bathroom      0
Car           0
Landsize      0
Year          0
Month         0
Crime       307
dtype: int64

In [54]:
# There are 307 instances where Crime has a NaN value, these rows will need to be dropped
combined_data_df = combined_data_df.dropna()
combined_data_df.head()

Suburb  Rooms Type      Price  Distance  Bathroom  Car  Landsize  \
1 0  abbotsford      2    h  1480000.0       2.5       1.0  1.0     202.0   
  1  abbotsford      2    h  1035000.0       2.5       1.0  0.0     156.0   
  2  abbotsford      4    h  1600000.0       2.5       1.0  2.0     120.0   
  3  abbotsford      2    h   941000.0       2.5       1.0  0.0     181.0   
  4  abbotsford      3    h  1876000.0       2.5       2.0  0.0     245.0   

     Year  Month   Crime  
1 0  2016      2  1053.0  
  1  2016      3  1053.0  
  2  2016      3  1053.0  
  3  2016      6  1053.0  
  4  2016      6  1053.0

In [55]:
print(f"The length of the combined dataframe is {len(combined_data_df.index)} rows.")

The length of the combined dataframe is 17394 rows.


#### Final edit

In [56]:
# Dropping suburb column now it is no longer needed
combined_data_df = combined_data_df.drop(columns=['Suburb'])
combined_data_df.head()

Rooms Type      Price  Distance  Bathroom  Car  Landsize  Year  Month  \
1 0      2    h  1480000.0       2.5       1.0  1.0     202.0  2016      2   
  1      2    h  1035000.0       2.5       1.0  0.0     156.0  2016      3   
  2      4    h  1600000.0       2.5       1.0  2.0     120.0  2016      3   
  3      2    h   941000.0       2.5       1.0  0.0     181.0  2016      6   
  4      3    h  1876000.0       2.5       2.0  0.0     245.0  2016      6   

      Crime  
1 0  1053.0  
  1  1053.0  
  2  1053.0  
  3  1053.0  
  4  1053.0

#### Exporting

In [57]:
combined_data_df.to_csv('output_data/clean_house_crime_data.csv', index = False)